(C) 2023-24 Manfred Schlaegl <manfred.schlaegl@jku.at>, Institute for Complex Systems, JKU Linz

SPDX-License-Identifier: BSD 3-clause "New" or "Revised" License

In [ ]:
import os
import rvvts

## Config

In [ ]:
%run config_internal.ipynb
%run config_host.ipynb

In [ ]:
# MAIN SETTINGS
instance = 0

dut = "VP"
#dut = "QEMU"

xlen = 32
#xlen = 64
    
#vector_vlen = 128
vector_vlen = 512


# build config
config = dict()
config.update(config_internal.copy())
config.update(config_host.copy())
config.update(dict(
    dir = os.getcwd() + "/run/FuzzCodeErrMinRunner_" + str(instance) + "_" + dut,

    ArchiveRunner_dut = rvvts.FuzzCodeErrMinRunner,
    
    debug_port = 8000 + instance,

    CompareRunner_dut = rvvts.DuTGDBRunner,

    # Optional: Coverage
    #RefCovRunner_coverage = rvvts.RISCVOVPSIMCoverageRunner,
    #RISCVOVPSIMCover_metric = "basic", # basic, extended or mnemonic
    #RISCVOVPSIMCover_sum_enable = True,
    
    xlen = xlen,
    vector_vlen = vector_vlen,
))

if dut == "VP":
    config["DuTGDBRunner_dut"] = rvvts.VPRunner
elif dut == "QEMU":
    config["DuTGDBRunner_dut"] = rvvts.QEMURunner
else:
    raise Exception("invalid dut")

## Setup

In [ ]:
runner = rvvts.ArchiveRunner(config)

## First Test

In [ ]:
#res = runner.run(min_fragments = 10, max_fragments = 100, blocking = True, timeout = 2.0)
#res = runner.run(min_fragments = 10000, max_fragments = 10000, blocking = True, timeout = 2.0)
res = runner.run(min_fragments = 5000, max_fragments = 5000, blocking = True, timeout = 2.0)

print(res[0])
print(res[1])

In [ ]:
# Print code (in case of error)
#print(len(runner.ArchiveRunner_dut.res_code_block.as_code()))
#print(runner.ArchiveRunner_dut.res_code_block.as_code())

## Manual Experiments

In [ ]:
# Manually generated test-code, or generated code (e.g. run/FuzzCodeErrMinRunner_0_VP/ArchiveRunner_xxx/ERROR_xxxxxx_iteration_xxxxxxxxxx/CodeErrMinRunner_0/CodeCompareRunner_0/BuildRunner_0/code.S)
code = """
"""

In [ ]:
#ret = runner.ArchiveRunner_dut.codecomparerunner.run(timeout = 2.0, blocking = True, code = runner.ArchiveRunner_dut.code.as_code())
ret = runner.ArchiveRunner_dut.codeerrminrunner.codecomparerunner.run(timeout = 2.0, blocking = True, code = code)
#print(code)
print(ret[0])
print(ret[1])

## Automated Experiments

In [ ]:
def FuzzCodeErrMin_stats(runner):
    cemr = runner.ArchiveRunner_dut.codeerrminrunner
    covr = cemr.codecomparerunner.compare_runner.CompareRunner_refcov.RefCovRunner_cov

    out = ", SUM: [ tests: " + str(cemr.tests) +\
            ", completes: " + str(cemr.completes) +\
            ", ignores: " + str(cemr.ignores) +\
            ", timeouts: " + str(cemr.timeouts) +\
            ", unknown_faults: " + str(cemr.unknown_faults) +\
            ", errors: " + str(cemr.errors) +\
            ", reductions: " + str(cemr.reductions) +\
            ", minimizations: " + str(cemr.minimizations) +\
            ", instr_errors: " + str(cemr.instr_errors)

    if covr:
        coverage_type = covr.coverage["sum"]["coverage"]["type"]
        coverage = covr.coverage["sum"]["coverage"]["percent"]
        out += ", coverage (" + str(coverage_type) + "): " + str(coverage) + "%"

    out += " ] "
    return out

In [ ]:
rvvts.runner_bench(
    runner,
    custom_stat_f = FuzzCodeErrMin_stats,
    iter = 10,
    min_fragments = 5000,
    max_fragments = 5000,
    timeout = 3.0,
    #stop_on_error = True,
)